# Ansible からのテスト

Ansible では様々なモジュールを使うことで様々な作業を自動で行うことができます。そして Ansible のモジュールは冪等性の担保のために「宣言した状態になっているか？」を事前に確認するため、ある意味でテストも一緒に行われているとも考えることができます。

しかし、実際に自動化を動かした際には、「本当に設定が入って、自分の想定通りになっているか？」という不安は常につきまといます。

また、他のケースでも、作業ではなくテストをAnsibleから網羅的に実施したいという状況にも多々遭遇します。

自動化において、この様な「不安をテストする」というのは非常に重要です。

ここでは、Ansible を使って、テストを記述する方法を解説してきます。

## テストに使えるモジュール

Ansible ではテストに使えるモジュールが各種用意されています。通常、モジュールは何らかの設定作業や変更作業を行いますが、ここで紹介するモジュールは、状態を取得する、状態をテストするためのものになります。

* [setup](http://docs.ansible.com/ansible/latest/setup_module.html) ・・・実行すると、作業対象サーバー（Linux系サーバーのみ）の設定情報を取得して、JSON形式で返します。Playbookの設定で gather_facts: true に設定されている場合、Playbookの実行前に自動的に処理が実行されます。
* [assert](http://docs.ansible.com/ansible/latest/assert_module.html) ・・・様々な条件を与えて、その条件を満たさない場合はエラーとして処理を停止します。
* [fail](http://docs.ansible.com/ansible/latest/fail_module.html) ・・・ assert の逆で、条件を満たした場合に失敗となりエラーで処理が停止します。
* [wait_for](http://docs.ansible.com/ansible/latest/wait_for_module.html) ・・・ファイルの作成・削除やポートのOpen/Closeを一定時間待ちます。設定時間を超えても指定した状態にならない場合にはエラーで処理が停止します。
* [shell](http://docs.ansible.com/ansible/latest/shell_module.html) ・・・一般的なシェルを実行して、その結果を取得します（戻り値や標準出力の結果を assert に渡すことができます）
* [set_fact](http://docs.ansible.com/ansible/latest/set_fact_module.html) ・・・変数に値をセットします。値を加工して使う場合によく利用します。

## Jinja2 フィルター

Ansible には、Ansibleが出力する様々な値に対してフィルターを行い、値の抽出や整形、判定を行う事ができます。ここでは Jinja2 形式のフィルターが利用できます。

Playbook の中では以下の形式で利用できます。

`"{{ result | successed }}"`

これは、モジュールの実行結果を result という変数に格納し、その変数 result に フィルタ「successed」を適用しています。このフィルタは格納された値の中からリターンコードが0であれば真を返します。逆に 「failed」 フィルターはリターンコードが0以外であれば真となります。

他にもAnsibleには様々なフィルターが存在しています。

[Filters](http://docs.ansible.com/ansible/latest/playbooks_filters.html)

演習ではこれらの一部を利用してテストを実施していきます。

## サンプルテストの実施

演習2で作成した環境をテストするPlaybookのサンプルが準備されているので、こちらを実行してテストを実行してください。

[server_ops_httpd_on_test.yml](/edit/aitac-automation-handson/server_ops_httpd_on_test.yml)

少し複雑に見るかかもしれませんが、テストしている内容はいたってシンプルです。まずは、実行してみてください。

In [ ]:
# この Cell を実行します。

ansible-playbook -i inventories/02_server_ops \
                 --private-key aitac-automation-key.pem -u centos -b \
                 server_ops_httpd_on_test.yml

ここまでの演習が問題なければ、正常に終了するはずです。

では、わざとエラーとなるように、HTTPDを停止して再度Playbookを実行してください。

In [ ]:
# この Cell を実行します。
ansible target -i inventories/02_server_ops \
               --private-key aitac-automation-key.pem -u centos -b \
               -m shell -a "systemctl stop httpd"

In [ ]:
# この Cell を実行します。

ansible-playbook -i inventories/02_server_ops \
                 --private-key aitac-automation-key.pem -u centos -b \
                 server_ops_httpd_on_test.yml

今度はエラーになったはずです。

HTTPDを再開して再度テストを実行してください。

In [ ]:
# この Cell を実行します。
ansible target -i inventories/02_server_ops \
               --private-key aitac-automation-key.pem -u centos -b \
               -m shell -a "systemctl start httpd"

In [ ]:
# この Cell を実行します。

ansible-playbook -i inventories/02_server_ops \
                 --private-key aitac-automation-key.pem -u centos -b \
                 server_ops_httpd_on_test.yml

再びテストが成功するようになったはずです。

ここで行われているテストは以下となります。

* rpm -qa の結果に httpd が含まれているかを確認 → 含まれていればパッケージがインストールされている
* ps -ef の結果に httpd が含まれているかを確認 → 含まれていればhttpdが起動している
* 他のホストから target サーバーのポート80がACTIVEになっているかを確認
* index.html が存在しているか？
* 他のホストから target サーバーへ接続して、コンテンツが取得できるか？
* 取得したコンテンツに特定の文字列が含まれているか？

サンプルですので、やや冗長なテストを実施していますが、Ansible からこのようなテストを実施することが可能です。



一度記述したテストは何度も実行可能です。従来は構築や何らかの変更作業の後に、様々な確認を人手で行っていたとおもいますが、Ansibleではそのようなテストも自動化し、何度もコスト0で実行することが可能です。

以下は、ダイナミックインベントリを利用した場合の例です。

[server_ops_httpd_on_test_with_dinv.yml](/edit/aitac-automation-handson/server_ops_httpd_on_test_with_dinv.yml)

Playbook の中で指定している `hosts:` 部分だけが異なっています。

In [ ]:
# この Cell を実行します。

ansible-playbook -i ./inventories/ec2.py \
                 -u centos --private-key aitac-automation-key.pem \
                 server_ops_httpd_on_test_with_dinv.yml

結果は通常のインベントリを使用した場合と同様になるはずです。

本演習は以上です。